<a href="https://colab.research.google.com/github/bs3537/DS-Unit-4-Sprint-1-NLP/blob/master/_Lower_case%2B_Bidirectional__LSTM_Keras_embeddings_NLP_whiskey.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np


In [0]:
#USING Keras embeddings and LSTM deep learning model: 
#DATASET: WHISKY RATINGS CLASSIFICATION FROM REVIEWS

In [0]:
import pandas as pd
train = pd.read_csv('https://raw.githubusercontent.com/bs3537/DS-Unit-4-Sprint-1-NLP/master/module3-document-classification/train.csv')

In [4]:
train.head(1)

,id,description,ratingCategory
0,1321,"\nSometimes, when whisky is batched, a few lef...",1


In [0]:
train['text'] = train['description'].str.strip('\n')

In [0]:
train = train.drop(columns=['description', 'id'])

In [0]:
train = train.rename(columns={"ratingCategory": "label"})

In [0]:
train['text'] = train['text'].str.lower()

In [9]:
train.head(1)

,label,text
0,1,"sometimes, when whisky is batched, a few lefto..."


In [10]:
train['label'].unique()

array([1, 0, 2])

In [11]:
from sklearn.model_selection import train_test_split

df_trn, df_val = train_test_split(train, stratify = train['label'], test_size = 0.10, random_state=42)

df_trn.shape, df_val.shape

((3678, 2), (409, 2))

In [0]:
#Define train and val datasets and X, Y vectors

In [0]:
X_train = df_trn['text'].values
y_train = df_trn['label'].values
X_val = df_val['text'].values
y_val = df_val['label'].values


In [0]:
#Tokenize using Keras 

In [0]:
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer

In [0]:
tokenizer_obj = Tokenizer()
total_reviews = df_trn['text'].values
tokenizer_obj.fit_on_texts(total_reviews)

In [0]:
#pad sequences

from tensorflow.keras.preprocessing.sequence import pad_sequences

In [0]:
max_length = max([len(s.split()) for s in total_reviews])

In [0]:
#define vocabulary size

vocab_size = len(tokenizer_obj.word_index) + 1

In [0]:
#define X_train and X_val tokens

X_train_tokens = tokenizer_obj.texts_to_sequences(X_train)

X_val_tokens = tokenizer_obj.texts_to_sequences(X_val)


In [0]:
#Define X_train and X_val pads

X_train_pad = pad_sequences(X_train_tokens, maxlen = max_length, padding='post')

X_val_pad = pad_sequences(X_val_tokens, maxlen = max_length, padding='post')

In [22]:
X_train_pad

array([[    8,     7,   952, ...,     0,     0,     0],
       [ 5632,  5633,  5634, ...,     0,     0,     0],
       [    3,  1098,   557, ...,     0,     0,     0],
       ...,
       [ 2378,  3942,    28, ...,     0,     0,     0],
       [  216,     6,  1121, ...,     0,     0,     0],
       [    8, 13991,    25, ...,     0,     0,     0]], dtype=int32)

In [0]:
#from sklearn.preprocessing import StandardScaler
#scaler = StandardScaler()

#X_train_scaled = scaler.fit_transform(X_train_pad)

#X_val_scaled = scaler.transform(X_val_pad)


In [0]:
#Build Keras Model

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout, Activation, SpatialDropout1D, Bidirectional
from tensorflow.keras.optimizers import Adam

In [0]:
#The Embedding layer requires the specification of the vocabulary size (vocab_size), the size of the real-valued vector space EMBEDDING_DIM = 100, and the maximum length of input documents max_length

In [0]:
model = Sequential()

In [0]:
#stacking layers on model


In [27]:
model.add(Embedding(input_dim=vocab_size, output_dim=64, input_length=max_length))
model.add(SpatialDropout1D(0.2))
model.add(Bidirectional(LSTM(64, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(4, activation='softmax')) 
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [28]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 237, 64)           895552    
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 237, 64)           0         
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               66048     
_________________________________________________________________
dense (Dense)                (None, 64)                8256      
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 260       
Total params: 970,116
Trainable params: 970,116
Non-trainable params: 0
__________________________________________________

In [29]:
model.fit(X_train_pad, y_train, batch_size=30, epochs=10, verbose=2, validation_data=(X_val_pad, y_val))

Epoch 1/10
123/123 - 168s - loss: 0.7505 - accuracy: 0.7012 - val_loss: 0.6764 - val_accuracy: 0.7042
Epoch 2/10
123/123 - 168s - loss: 0.6371 - accuracy: 0.7162 - val_loss: 0.5972 - val_accuracy: 0.7090
Epoch 3/10
123/123 - 168s - loss: 0.4712 - accuracy: 0.8157 - val_loss: 0.6434 - val_accuracy: 0.7066
Epoch 4/10
123/123 - 168s - loss: 0.2992 - accuracy: 0.8899 - val_loss: 0.8686 - val_accuracy: 0.6748
Epoch 5/10
123/123 - 167s - loss: 0.1813 - accuracy: 0.9459 - val_loss: 1.0025 - val_accuracy: 0.6822
Epoch 6/10
123/123 - 168s - loss: 0.1162 - accuracy: 0.9644 - val_loss: 1.0495 - val_accuracy: 0.6919
Epoch 7/10
123/123 - 168s - loss: 0.0866 - accuracy: 0.9717 - val_loss: 1.2866 - val_accuracy: 0.6724
Epoch 8/10
123/123 - 168s - loss: 0.0477 - accuracy: 0.9837 - val_loss: 1.4575 - val_accuracy: 0.6797
Epoch 9/10
123/123 - 167s - loss: 0.0509 - accuracy: 0.9826 - val_loss: 1.4279 - val_accuracy: 0.6699
Epoch 10/10
123/123 - 167s - loss: 0.0355 - accuracy: 0.9869 - val_loss: 1.7607 - 